In [14]:
# 0) CONFIG
# =======================
DATA_PATH = 'alldemand_augjul.csv'

FORECAST_HORIZON = 7
FORECAST_START_DATE = None             # contoh '2025-08-01'; None = H+1 dari histori terakhir
FORECAST_START_OFFSET_DAYS = 1

FORECAST_SITE_CODES = ['IEL-ST-KDI']             # contoh: ['IEL-MU-SFI', 'IEL-KS-ANGSANA']
TRAIN_SITE_CODES    = None             # batasi data training ke site tertentu (opsional)

ROUNDING_MODE = 'half_up'              # 'half_up' | 'round' | 'ceil' | 'floor'
ZERO_THR = 0.5                         # prediksi < ZERO_THR dianggap 0 (tuning 0.1–1.0)

RANDOM_STATE = 42
DAYFIRST = True                        # parsing tanggal day-first (format umum ID)

In [2]:
# 1) IMPORTS + HELPERS
# =======================
import os, time
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import timedelta

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge

# TransformedTargetRegressor (fallback utk sklearn lama)
try:
    from sklearn.compose import TransformedTargetRegressor as TTR
except Exception:
    class TTR:
        def __init__(self, regressor, func=None, inverse_func=None):
            self.regressor = regressor
            self.func = func if func else (lambda x: x)
            self.inverse_func = inverse_func if inverse_func else (lambda x: x)
        def fit(self, X, y):
            self.regressor.fit(X, self.func(np.asarray(y))); return self
        def predict(self, X):
            return self.inverse_func(self.regressor.predict(X))

def _rmse(y, yhat):
    try: return mean_squared_error(y, yhat, squared=False)
    except TypeError: return np.sqrt(mean_squared_error(y, yhat))

def smape(y, yhat):
    y = np.asarray(y, float); yhat = np.asarray(yhat, float)
    d = np.abs(y) + np.abs(yhat); d = np.where(d==0, 1.0, d)
    return np.mean(2*np.abs(yhat - y)/d)   # 0–2; kalikan 100 untuk %

def mape(y, yhat):
    y = np.asarray(y, float); yhat = np.asarray(yhat, float)
    denom = np.where(y==0, 1.0, y)
    return np.mean(np.abs((y - yhat)/denom))  # 0–∞; kalikan 100 untuk %

def wape(y_true, y_pred):
    y_true = np.asarray(y_true, float); y_pred = np.asarray(y_pred, float)
    den = np.sum(np.abs(y_true)); den = den if den != 0 else 1.0
    return 100 * np.sum(np.abs(y_true - y_pred)) / den

def mase(y_true, y_pred, insample):
    from sklearn.metrics import mean_absolute_error
    denom = mean_absolute_error(insample[1:], insample[:-1]) if len(insample) > 1 else 1.0
    return mean_absolute_error(y_true, y_pred) / (denom if denom != 0 else 1.0)

def metrics(y, yhat, as_percent=False):
    m = dict(MAE=mean_absolute_error(y, yhat),
             RMSE=_rmse(y, yhat),
             sMAPE=smape(y, yhat),
             MAPE=mape(y, yhat))
    if as_percent:
        m['sMAPE'] *= 100.0
        m['MAPE']  *= 100.0
    return m

def print_metrics(m):
    for k,v in m.items(): print(f"{k}: {v:.4f}")

def eval_with_rounding(y_true, y_pred, thr=0.5):
    x = np.asarray(y_pred, float)
    x = np.where(x < thr, 0, x)                 # threshold -> nol
    x = np.floor(x + 0.5)                       # half-up rounding
    return {
        'MAE': mean_absolute_error(y_true, x),
        'RMSE': _rmse(y_true, x),
        'sMAPE%': smape(y_true, x) * 100,
        'MAPE%': mape(y_true, x) * 100,
        'WAPE%': wape(y_true, x),
    }

def safe_save_csv(df, path):
    p = Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    try:
        tmp = p.with_suffix(p.suffix + '.tmp')
        df.to_csv(tmp, index=False, encoding='utf-8-sig'); os.replace(tmp, p)
        print(f"Saved: {p}")
    except PermissionError:
        alt = p.with_name(p.stem + f'_{int(time.time())}' + p.suffix)
        df.to_csv(alt, index=False, encoding='utf-8-sig'); print(f"Target locked. Saved: {alt}")

def add_calendar_features(df):
    df = df.copy()
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    return df

def add_group_lags_rolls(df, group_cols, target_col='demand_qty',
                         lags=(1,7,14,28), roll_windows=(7,14,28)):
    df = df.sort_values(group_cols + ['date']).reset_index(drop=True)
    g = df.groupby(group_cols, group_keys=False)
    for L in lags: df[f'lag_{L}'] = g[target_col].shift(L)
    for W in roll_windows: df[f'rollmean_{W}'] = g[target_col].shift(1).rolling(W).mean()
    return df

def complete_calendar_daily(df, group_cols=('partnumber','site_code'), target='demand_qty'):
    out = []
    for keys, g in df.groupby(list(group_cols), sort=False):
        gd = (g.groupby('date', as_index=False)[target].sum().sort_values('date'))
        idx = pd.date_range(gd['date'].min(), gd['date'].max(), freq='D')
        gg = gd.set_index('date').reindex(idx).rename_axis('date').reset_index()
        gg[target] = pd.to_numeric(gg[target], errors='coerce').fillna(0)
        if not isinstance(keys, tuple): keys = (keys,)
        for c, v in zip(group_cols, keys): gg[c] = v
        out.append(gg)
    return pd.concat(out, ignore_index=True)

def round_series(x, mode='half_up'):
    if mode == 'half_up': return np.floor(x + 0.5).astype(int)
    if mode == 'round':   return np.round(x).astype(int)
    if mode == 'ceil':    return np.ceil(x).astype(int)
    if mode == 'floor':   return np.floor(x).astype(int)
    return np.round(x).astype(int)

def make_ohe(dense=False):
    try:    return OneHotEncoder(handle_unknown='ignore', sparse_output=not dense)
    except TypeError: return OneHotEncoder(handle_unknown='ignore', sparse=not dense)

def robust_read_table(path):
    if path.lower().endswith(('.xlsx','.xls')):
        return pd.read_excel(path, dtype=str)
    for enc in ('utf-8-sig','cp1252','latin1','iso-8859-1'):
        try:
            df_ = pd.read_csv(path, sep=None, engine='python', dtype=str, encoding=enc)
            print(f"Loaded with encoding={enc}"); return df_
        except Exception:
            continue
    # fallback
    return pd.read_csv(path, sep=None, engine='python', dtype=str, encoding='latin1', errors='replace')


In [3]:
# 2) LOAD & NORMALIZE
# =======================
df = robust_read_table(DATA_PATH)
df.columns = [c.strip().lower() for c in df.columns]
req = {'demand_qty','date','partnumber','site_code'}
miss = req - set(df.columns)
if miss: raise ValueError(f"Kolom wajib hilang: {miss}. Header: {list(df.columns)}")

df['partnumber'] = df['partnumber'].astype(str).str.strip()
df['site_code']  = df['site_code'].astype(str).str.strip()
d = pd.to_datetime(df['date'], dayfirst=DAYFIRST, errors='coerce')
if d.isna().mean() > 0.2:
    d = pd.to_datetime(df['date'], dayfirst=not DAYFIRST, errors='coerce')
df['date'] = d
if df['date'].isna().any():
    raise ValueError("Ada tanggal gagal parse. Cek format kolom 'date'.")

df['demand_qty'] = pd.to_numeric(df['demand_qty'], errors='coerce').fillna(0)
df = df.sort_values(['partnumber','site_code','date']).reset_index(drop=True)

Loaded with encoding=utf-8-sig


In [4]:
# 3) AGG DAILY + CALENDAR + CLAMP
# =======================
df = (df.groupby(['partnumber','site_code','date'], as_index=False)
        .agg(demand_qty=('demand_qty','sum')))
df = complete_calendar_daily(df, group_cols=('partnumber','site_code'), target='demand_qty')
p99 = df.groupby(['partnumber','site_code'])['demand_qty'].transform(lambda s: s.quantile(0.99))
df['demand_qty'] = df['demand_qty'].clip(lower=0, upper=p99)

In [5]:
# =======================
# 4) TRAINING SUBSET (opsional)
# =======================
df_model = df if TRAIN_SITE_CODES is None else df[df['site_code'].isin(TRAIN_SITE_CODES)].copy()
if TRAIN_SITE_CODES is not None and df_model.empty:
    raise ValueError("TRAIN_SITE_CODES tidak ditemukan di data.")

In [6]:
# 5) FEATURE ENGINEERING
# =======================
group_cols = ['partnumber','site_code']
df_fe = add_calendar_features(df_model)
df_fe = add_group_lags_rolls(df_fe, group_cols, target_col='demand_qty',
                             lags=(1,7,14,28), roll_windows=(7,14,28))
need = [c for c in df_fe.columns if c.startswith('lag_') or c.startswith('rollmean_')]
df_fe = df_fe[df_fe[need].notnull().all(axis=1)].reset_index(drop=True)

feature_cols_cat = ['partnumber','site_code']
feature_cols_num = ['year','month','day','dayofweek','weekofyear','is_month_start','is_month_end'] + need


In [7]:
# 6) SPLIT & BASELINES
# =======================
cutoff = df_fe['date'].max() - pd.Timedelta(days=max(28, 2*FORECAST_HORIZON))
train = df_fe[df_fe['date'] <= cutoff].copy()
valid = df_fe[df_fe['date'] >  cutoff].copy()

X_train = pd.concat([train[feature_cols_cat], train[feature_cols_num]], axis=1)
X_valid = pd.concat([valid[feature_cols_cat], valid[feature_cols_num]], axis=1)
y_train = train['demand_qty'].astype(float).values
y_valid = valid['demand_qty'].astype(float).values

print("Baseline: Naive t-1 (raw)")
print_metrics(metrics(y_valid, valid.get('lag_1', 0).fillna(0).values, as_percent=True))
print("\nBaseline: Naive t-7 (raw)")
print_metrics(metrics(y_valid, valid.get('lag_7', 0).fillna(0).values, as_percent=True))

Baseline: Naive t-1 (raw)
MAE: 1.6807
RMSE: 34.9450
sMAPE: 20.9861
MAPE: 48.7361

Baseline: Naive t-7 (raw)
MAE: 1.6625
RMSE: 34.8148
sMAPE: 20.5459
MAPE: 47.1213


In [8]:
# 7) PREPROCESS + CANDIDATE MODELS
# =======================
preprocess_sparse = ColumnTransformer(
    transformers=[('cat', make_ohe(dense=False), feature_cols_cat)],
    remainder='passthrough'
)
preprocess_dense = ColumnTransformer(
    transformers=[('cat', make_ohe(dense=True), feature_cols_cat)],
    remainder='passthrough'
)

CANDIDATES = {
    "RF_log": Pipeline([
        ("prep", preprocess_sparse),
        ("reg", TTR(
            regressor=RandomForestRegressor(
                n_estimators=800, max_depth=None, min_samples_leaf=2,
                n_jobs=-1, random_state=RANDOM_STATE
            ),
            func=np.log1p, inverse_func=np.expm1
        ))
    ]),
    "ET_log": Pipeline([
        ("prep", preprocess_sparse),
        ("reg", TTR(
            regressor=ExtraTreesRegressor(
                n_estimators=800, max_depth=None, min_samples_leaf=2,
                n_jobs=-1, random_state=RANDOM_STATE
            ),
            func=np.log1p, inverse_func=np.expm1
        ))
    ]),
    "GBR_log": Pipeline([
        ("prep", preprocess_dense),
        ("reg", TTR(
            regressor=GradientBoostingRegressor(random_state=RANDOM_STATE),
            func=np.log1p, inverse_func=np.expm1
        ))
    ]),
    "Ridge_log": Pipeline([
        ("prep", preprocess_dense),
        ("reg", TTR(
            regressor=Ridge(alpha=1.0),
            func=np.log1p, inverse_func=np.expm1
        ))
    ]),
}

In [9]:
# 8) TRAIN, EVALUATE, PICK BEST
# =======================
results, fitted = {}, {}

for name, est in CANDIDATES.items():
    est.fit(X_train, y_train)
    y_pred = est.predict(X_valid)

    # raw metrics
    m_raw = metrics(y_valid, y_pred, as_percent=True)
    print(f"\n{name} (raw):"); print_metrics(m_raw)
    print(f"WAPE% (raw): {wape(y_valid, y_pred):.4f}  MASE: {mase(y_valid, y_pred, y_train):.4f}")

    # rounded + threshold metrics
    m_rnd = eval_with_rounding(y_valid, y_pred, thr=ZERO_THR)
    print(f"{name} (rounded + thr={ZERO_THR}):")
    for k,v in m_rnd.items(): print(f"{k}: {v:.4f}")

    results[name] = m_raw  # gunakan RMSE raw untuk pemilihan model
    fitted[name]  = est

best_name = min(results, key=lambda k: results[k]["RMSE"])
best_est = fitted[best_name]
print(f"\n== Best model by RMSE (raw): {best_name} ==")


RF_log (raw):
MAE: 1.3337
RMSE: 34.0131
sMAPE: 173.2997
MAPE: 13.3423
WAPE% (raw): 103.3689  MASE: 2.2015
RF_log (rounded + thr=0.5):
MAE: 1.3082
RMSE: 34.0154
sMAPE%: 20.8042
MAPE%: 10.5495
WAPE%: 101.3923

ET_log (raw):
MAE: 1.3337
RMSE: 34.0099
sMAPE: 169.9400
MAPE: 13.3074
WAPE% (raw): 103.3700  MASE: 2.2016
ET_log (rounded + thr=0.5):
MAE: 1.3098
RMSE: 34.0100
sMAPE%: 20.2248
MAPE%: 10.7165
WAPE%: 101.5161

GBR_log (raw):
MAE: 1.3265
RMSE: 34.0167
sMAPE: 199.0608
MAPE: 12.2918
WAPE% (raw): 102.8113  MASE: 2.1897
GBR_log (rounded + thr=0.5):
MAE: 1.2950
RMSE: 34.0206
sMAPE%: 17.4356
MAPE%: 8.8996
WAPE%: 100.3713

Ridge_log (raw):
MAE: 1.3269
RMSE: 34.0153
sMAPE: 199.2902
MAPE: 12.3770
WAPE% (raw): 102.8384  MASE: 2.1902
Ridge_log (rounded + thr=0.5):
MAE: 1.2938
RMSE: 34.0208
sMAPE%: 17.5820
MAPE%: 8.7962
WAPE%: 100.2785

== Best model by RMSE (raw): ET_log ==


In [ ]:
# === Kumpulkan metrik ke DataFrame & tampilkan penuh ===
from IPython.display import display, HTML

rows = []
for name, est in fitted.items():        # fitted = dict model terlatih dari langkahmu
    y_pred = est.predict(X_valid)

    # raw
    m_raw = metrics(y_valid, y_pred, as_percent=True)
    rows.append({
        "model": name, "eval": "raw",
        "MAE": m_raw["MAE"], "RMSE": m_raw["RMSE"],
        "sMAPE%": m_raw["sMAPE"], "MAPE%": m_raw["MAPE"],
    })

    # rounded + threshold
    m_rnd = eval_with_rounding(y_valid, y_pred, thr=ZERO_THR)  # ZERO_THR dari config
    rows.append({
        "model": name, "eval": f"rounded(thr={ZERO_THR})",
        **m_rnd
    })

results_df = pd.DataFrame(rows)
results_df = results_df[
    ["model","eval","MAE","RMSE","sMAPE%","MAPE%"]
].sort_values(["eval","RMSE"], ascending=[True, True])

# Atur tampilan agar tidak terpotong
pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 0)           # lebar fleksibel
pd.set_option("display.float_format", lambda x: f"{x:.4f}")

display(HTML(results_df.to_html(index=False)))  



model,eval,MAE,RMSE,sMAPE%,MAPE%
ET_log,raw,1.3337,34.0099,169.9400,13.3074
RF_log,raw,1.3337,34.0131,173.2997,13.3423
Ridge_log,raw,1.3269,34.0153,199.2902,12.3770
GBR_log,raw,1.3265,34.0167,199.0608,12.2918
ET_log,rounded(thr=0.5),1.3098,34.0100,20.2248,10.7165
RF_log,rounded(thr=0.5),1.3082,34.0154,20.8042,10.5495
GBR_log,rounded(thr=0.5),1.2950,34.0206,17.4356,8.8996
Ridge_log,rounded(thr=0.5),1.2938,34.0208,17.5820,8.7962


In [ ]:
# === 1) Pilih best model berdasar MAPE% terkecil ===
eval_target = f"rounded(thr={ZERO_THR})"
subset = results_df[results_df["eval"] == eval_target]
if subset.empty:                             
    eval_target = "raw"
    subset = results_df[results_df["eval"] == eval_target]

best_row  = subset.sort_values("MAPE%").iloc[0]
best_name = best_row["model"]
best_est  = fitted[best_name]
print(f"Best model by MAPE% on {eval_target}: {best_name} | MAPE%={best_row['MAPE%']:.4f}")

# === 2) Siapkan subset site untuk OUTPUT (training tetap dari df) ===
df_sites = df if 'FORECAST_SITE_CODES' not in globals() or (FORECAST_SITE_CODES is None) \
              else df[df['site_code'].isin(FORECAST_SITE_CODES)].copy()
if df_sites.empty:
    raise ValueError("FORECAST_SITE_CODES tidak ditemukan di data.")

# === 3) Satu-hari forecast helper ===
def one_day_forecast(history_df, fdate):
    # Fitur terakhir dari histori
    hist = add_calendar_features(history_df)
    hist = add_group_lags_rolls(hist, group_cols, target_col='demand_qty',
                                lags=(1,7,14,28), roll_windows=(7,14,28))
    latest = (hist.sort_values('date')
                .groupby(group_cols, as_index=False)
                .tail(1)[[*group_cols] + [c for c in hist.columns
                                          if c.startswith('lag_') or c.startswith('rollmean_')]])

    # Kombinasi (part, site) di tanggal fdate
    combos = df_sites[group_cols].drop_duplicates().reset_index(drop=True)
    combos['date'] = fdate
    combos = add_calendar_features(combos).merge(latest, on=group_cols, how='left')

    # Isi NaN untuk fitur lag/rolling
    lagroll_cols = [c for c in feature_cols_num if c.startswith('lag_') or c.startswith('rollmean_')]
    for c in lagroll_cols:
        if c in combos:
            combos[c] = combos[c].fillna(0)

    # Prediksi → threshold → pembulatan
    Xf = pd.concat([combos[feature_cols_cat], combos[feature_cols_num]], axis=1)
    raw_model = np.maximum(0, best_est.predict(Xf))              # mentah dari model
    raw_thr   = np.where(raw_model < ZERO_THR, 0, raw_model)     # setelah threshold
    combos['forecast_qty_raw_model'] = raw_model
    combos['forecast_qty_raw']       = raw_thr
    combos['forecast_qty']           = round_series(raw_thr, ROUNDING_MODE)
    combos['date'] = fdate

    return combos[['date','partnumber','site_code',
                   'forecast_qty','forecast_qty_raw','forecast_qty_raw_model']]

# === 4) Tentukan tanggal mulai & histori sampai H-1 ===
max_hist   = df_sites['date'].max()
start_date = pd.to_datetime(FORECAST_START_DATE) if FORECAST_START_DATE \
             else (max_hist + pd.Timedelta(days=FORECAST_START_OFFSET_DAYS))

history = (df_sites[df_sites['date'] <= start_date - pd.Timedelta(days=1)].copy()
           if start_date <= max_hist else df_sites.copy())

# Warm-up kalau ada gap sebelum start_date
gap = history['date'].max() + pd.Timedelta(days=1)
while gap < start_date:
    tmp = one_day_forecast(history, gap)
    # feed-back menggunakan nilai setelah threshold
    history = pd.concat([history,
                         tmp.rename(columns={'forecast_qty_raw':'demand_qty'})
                           [['date','partnumber','site_code','demand_qty']]],
                        ignore_index=True)
    gap += pd.Timedelta(days=1)

# === 5) Forecast utama untuk horizon ===
forecasts = []
for d in pd.date_range(start_date, periods=FORECAST_HORIZON, freq='D'):
    out = one_day_forecast(history, d)
    forecasts.append(out)
    history = pd.concat([history,
                         out.rename(columns={'forecast_qty_raw':'demand_qty'})
                           [['date','partnumber','site_code','demand_qty']]],
                        ignore_index=True)

forecast_df = (pd.concat(forecasts, ignore_index=True)
                 .sort_values(['partnumber','site_code','date'])
                 .reset_index(drop=True))

print("\nSample forecast:")
display(forecast_df.head())


Best model by MAPE% on rounded(thr=0.5): Ridge_log | MAPE%=8.7962

Sample forecast:


,date,partnumber,site_code,forecast_qty,forecast_qty_raw,forecast_qty_raw_model
0,2025-08-01,06.11251.2005,IEL-ST-KDI,0,0.0000,0.0310
1,2025-08-02,06.11251.2005,IEL-ST-KDI,0,0.0000,0.0486
2,2025-08-03,06.11251.2005,IEL-ST-KDI,0,0.0000,0.0355
3,2025-08-04,06.11251.2005,IEL-ST-KDI,0,0.0000,0.0756
4,2025-08-05,06.11251.2005,IEL-ST-KDI,0,0.0000,0.0690


In [21]:
forecast_df.to_csv("forecast_best_by_mape.csv", index=False, encoding="utf-8-sig")

In [ ]:
# 9) FORECAST LOOP (threshold + keep raw_model)
# =======================
df_sites = df if FORECAST_SITE_CODES is None else df[df['site_code'].isin(FORECAST_SITE_CODES)].copy()
if df_sites.empty:
    raise ValueError("FORECAST_SITE_CODES tidak ditemukan di data.")

def one_day_forecast(history_df, fdate):
    hist = add_calendar_features(history_df)
    hist = add_group_lags_rolls(hist, group_cols, target_col='demand_qty',
                                lags=(1,7,14,28), roll_windows=(7,14,28))
    latest = (hist.sort_values('date')
                .groupby(group_cols, as_index=False)
                .tail(1)[[*group_cols] + [c for c in hist.columns
                                          if c.startswith('lag_') or c.startswith('rollmean_')]])

    combos = df_sites[group_cols].drop_duplicates().reset_index(drop=True)
    combos['date'] = fdate
    combos = add_calendar_features(combos).merge(latest, on=group_cols, how='left')

    lagroll_cols = [c for c in feature_cols_num if c.startswith('lag_') or c.startswith('rollmean_')]
    for c in lagroll_cols:
        if c in combos: combos[c] = combos[c].fillna(0)

    Xf = pd.concat([combos[feature_cols_cat], combos[feature_cols_num]], axis=1)
    raw_model = np.maximum(0, best_est.predict(Xf))              # prediksi mentah (disimpan)
    raw_thr   = np.where(raw_model < ZERO_THR, 0, raw_model)      # terapkan ambang nol

    combos['forecast_qty_raw_model'] = raw_model                  # simpan mentah model
    combos['forecast_qty_raw'] = raw_thr                          # setelah threshold
    combos['forecast_qty'] = round_series(raw_thr, ROUNDING_MODE) # hasil operasional
    combos['date'] = fdate

    return combos[['date','partnumber','site_code',
                   'forecast_qty','forecast_qty_raw','forecast_qty_raw_model']]

max_hist = df_sites['date'].max()
start_date = pd.to_datetime(FORECAST_START_DATE) if FORECAST_START_DATE else (
    max_hist + pd.Timedelta(days=FORECAST_START_OFFSET_DAYS)
)

history = df_sites[df_sites['date'] <= start_date - pd.Timedelta(days=1)].copy() \
           if start_date <= max_hist else df_sites.copy()

gap = history['date'].max() + pd.Timedelta(days=1)
while gap < start_date:
    tmp = one_day_forecast(history, gap)
    # feed-back menggunakan nilai setelah threshold 
    history = pd.concat([history,
                         tmp.rename(columns={'forecast_qty_raw':'demand_qty'})
                           [['date','partnumber','site_code','demand_qty']]],
                        ignore_index=True)
    gap += pd.Timedelta(days=1)

forecasts = []
for d in pd.date_range(start_date, periods=FORECAST_HORIZON, freq='D'):
    out = one_day_forecast(history, d)
    forecasts.append(out)
    history = pd.concat([history,
                         out.rename(columns={'forecast_qty_raw':'demand_qty'})
                           [['date','partnumber','site_code','demand_qty']]],
                        ignore_index=True)

forecast_df = (pd.concat(forecasts, ignore_index=True)
                 .sort_values(['partnumber','site_code','date'])
                 .reset_index(drop=True))

print("\nSample output:")
print(forecast_df.head())


Sample output:
        date partnumber site_code  forecast_qty  forecast_qty_raw  \
0 2025-08-01              Kendari             0               0.0   
1 2025-08-02              Kendari             0               0.0   
2 2025-08-03              Kendari             0               0.0   
3 2025-08-04              Kendari             0               0.0   
4 2025-08-05              Kendari             0               0.0   

   forecast_qty_raw_model  
0                0.000000  
1                0.016655  
2                0.000000  
3                0.000592  
4                0.000409  


In [15]:
# 10) SAVE
# =======================
outdir = Path('outputs_forecast11')
safe_save_csv(forecast_df, outdir / f'forecast_next_{FORECAST_HORIZON}d.csv')

if FORECAST_SITE_CODES is not None:
    sel = forecast_df[forecast_df['site_code'].isin(FORECAST_SITE_CODES)].copy()
    safe_save_csv(sel, outdir / f'forecast_selected_next_{FORECAST_HORIZON}d.csv')

Saved: outputs_forecast11\forecast_next_7d.csv
Saved: outputs_forecast11\forecast_selected_next_7d.csv
